In [66]:
from __future__ import print_function

In [67]:
#!jupyter nbconvert --to python Dataset.ipynb
from Dataset import GenericDataset

In [68]:
import cv2
import random
import torch
import torchvision.transforms as transforms
import torch.utils.data as data

In [69]:
import glob
import os
import shutil

In [70]:
class videoFrameError(Exception):
    """
        Raise when reading a non-existing frame in a video
        Attribute : 
            videoName --- video name or path
            frame     --- frame number
    """
    def __init__(self, videoName, frame):
        self.videoName = videoName
        self.frame = frame
    def __str__(self):
        return repr(self.videoName) + repr(self.frame)

In [71]:
def ReadRandomSequence(videoName, nbFrame, transform=transforms.ToTensor(), dropFrame=1):
    """
        Read a nbFrame number of frame randomly in the video
    """
    cap = cv2.VideoCapture(videoName)
    if cv2.CAP_PROP_FRAME_COUNT - (nbFrame*dropFrame) < 0:
        firstFrame=cv2.CAP_PROP_FRAME_COUNT
    else:
        firstFrame = random.randint(0, cap.get(cv2.CAP_PROP_FRAME_COUNT)- (nbFrame*dropFrame))
    
    ret = cap.set(cv2.CAP_PROP_POS_FRAMES,firstFrame) #go to first frame
    cframe = firstFrame #current frame

    if not ret:
        raise videoFrameError(videoName, cframe)
    t = torch.Tensor(nbFrame,3,225,225)
    for i in range(nbFrame):
        ret, frame = cap.read()
        cframe += 1
        if not ret:
            raise videoFrameError(videoName, cframe)    
        t[i] = transform(frame)
    return t

In [85]:
def addOpticalFlow(img, prevImg):
    print("Img shape : ", img.shape)
    print("prevImg shape : ", prevImg.shape)
    f = cv2.cvtColor(prevImg,cv2.COLOR_BGR2GRAY)
    nf = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    oF = cv2.calcOpticalFlowFarneback(f,nf, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    return np.dstack((img,oF))

In [73]:
def ReadNFrame(videoName, nbFrame, transform=None, dropFrame=0, opticalFlow=False):
    """
        Read a nbFrame frame and concatenate them
        
        Input : videoName: path to video
                nbFrame : number of frame to read
                transform : transform to apply to each frame
                dropFrame : number of frame to drop between each frame
        
    """
    cap = cv2.VideoCapture(videoName)
    cap.set(cv2.CAP_PROP_CONVERT_RGB, True)
    #verify video length
    if cap.get(cv2.CAP_PROP_FRAME_COUNT) < (nbFrame*(dropFrame+1)-dropFrame):
        print("Error in video Reading", videoName, nbFrame)
        print("This video only has ", cap.get(cv2.CAP_PROP_FRAME_COUNT), "frames")
        raise videoFrameError(videoName, nbFrame)
    
    #find possible first frame
    firstFrame = random.randint(0, cap.get(cv2.CAP_PROP_FRAME_COUNT)- (nbFrame*(dropFrame+1)-dropFrame))
    ret, frame = cap.read() #save for image shape
    ret = cap.set(cv2.CAP_PROP_POS_FRAMES,firstFrame) #go to first frame
    cframe = firstFrame #current frame

    if not ret:
        print("Error in video Reading", videoName, cframe)
        print("This video only has ", cap.get(cv2.CAP_PROP_FRAME_COUNT), "frames")
        raise videoFrameError(videoName, cframe)
        
    imBase = transform(frame)
    if opticalFlow:
        t = torch.Tensor(nbFrame,5,imBase.size(1),imBase.size(2))
    else:    
        t = torch.Tensor(nbFrame,3,imBase.size(1),imBase.size(2))
        
    for i in range(nbFrame):
        ret, frame = cap.read()
        if opticalFlow and i == 0:
            prevFrame = frame.copy()
        cframe += 1
        if not ret:
            raise videoFrameError(videoName, cframe)    
        
        if opticalFlow:
            frame = addOpticalFlow(frame, prevFrame)
            prevFrame = frame.copy()
        
        t[i] = transform(frame)
        j = 0
        while j < dropFrame:
            ret, frame = cap.read()
            j += 1
        
    return t.view(t.size(0)*t.size(1), t.size(2), t.size(3))

In [80]:
def VideoDataset(rep="/video/GestureSequence/", SequenceSize=5, batchSize=4, transform=None, 
            concat=False, dropFrame=1, opticalFlow=False):
    
    
    if concat:
        openFile = lambda x:ReadNFrame(x,SequenceSize,transform, dropFrame,opticalFlow=opticalFlow)
    else:
        openFile = lambda x:ReadRandomSequence(x,SequenceSize,transform, dropFrame, opticalFlow=opticalFlow)
    
    targetOperation = lambda x:int(x)
    
    videodataset = GenericDataset(rep, fileOperation=openFile, targetOperation=targetOperation)
    data_loader = data.DataLoader(videodataset,
                                          batch_size=batchSize,
                                          shuffle=True,
                                          num_workers=4,
                                          drop_last=True
                                )
    return data_loader

In [81]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np
import random

%matplotlib inline
def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    show(make_grid([a[0][0][:3], a[0][0][3:6], a[0][0][6:9]], padding=1, normalize=False))

In [76]:
def countFrame(fileName):
    cap = cv2.VideoCapture(fileName)
    return cap.get(cv2.CAP_PROP_FRAME_COUNT)

In [77]:
def moveShortVideos(rep="/video/GestureSequence/", trash="/video/VideoTrash"):
    import glob
    for f in glob.iglob(os.path.join(rep,'*')):
        if os.path.isdir(f):
            moveShortVideos(f, trash)
            print("Going into", f)
        else:
            if '.avi' in f:
                print("Handling", f, end='\r')
                nframe = countFrame(f)
                if nframe < 9:
                    print("Moving ", f,end="\t\n")
                    shutil.move(f, trash)

In [86]:
if __name__=="__main__":
    trans = transforms.Compose(
                (transforms.ToPILImage(),
                transforms.Resize(64),
                transforms.RandomCrop(64),
                transforms.ToTensor())
                )
    vd = VideoDataset(rep="/video/GestureSequence/", SequenceSize=3, transform=trans, batchSize=12, concat=True, opticalFlow=True)
    for i in vd:
        a = i
        break
    a
    #countFrame("/video/Gesture/gR16.mp4")
    #moveShortVideos()

Img shape :  (960, 1280, 3)
prevImg shape :  (960, 1280, 3)
Img shape :  (960, 1280, 3)
Img shape :  (960, 1280, 3)
prevImg shape :  (960, 1280, 3)
prevImg shape :  (960, 1280, 3)
Img shape :  (960, 1280, 3)
prevImg shape :  (960, 1280, 3)


KeyboardInterrupt: 